In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
os.getcwd()

'/home/jupyter'

In [6]:
!pip install peft>=0.11.1

In [4]:
!pip install -q -U git+https://github.com/huggingface/peft.git

ERROR: Ignored the following versions that require a different python version: 0.21.0 Requires-Python >=3.8.0; 0.22.0 Requires-Python >=3.8.0; 0.23.0 Requires-Python >=3.8.0; 0.24.0 Requires-Python >=3.8.0; 0.24.1 Requires-Python >=3.8.0; 0.25.0 Requires-Python >=3.8.0; 0.26.0 Requires-Python >=3.8.0; 0.26.1 Requires-Python >=3.8.0; 0.27.0 Requires-Python >=3.8.0; 0.27.1 Requires-Python >=3.8.0; 0.27.2 Requires-Python >=3.8.0; 0.28.0 Requires-Python >=3.8.0; 0.29.0 Requires-Python >=3.8.0; 0.29.1 Requires-Python >=3.8.0; 0.29.2 Requires-Python >=3.8.0; 0.29.3 Requires-Python >=3.8.0; 0.30.0 Requires-Python >=3.8.0; 0.30.0rc0 Requires-Python >=3.8.0; 0.30.1 Requires-Python >=3.8.0; 0.31.0 Requires-Python >=3.8.0
ERROR: Could not find a version that satisfies the requirement accelerate>=0.21.0 (from peft) (from versions: 0.0.1, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.4.0, 0.5.0, 0.5.1, 0.6.0, 0.6.1, 0.6.2, 0.7.0, 0.7.1, 0.8.0, 0.9.0, 0.10.0, 0.11.0, 0.12.0, 0.13.0, 0.13.1, 0.13.2, 0.14.0, 0.15.0, 

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import seaborn as sns
import matplotlib.pyplot as plt
from accelerate import FullyShardedDataParallelPlugin, Accelerator
#from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
from datasets import Dataset
from transformers import AutoTokenizer
#from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
#from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
import transformers
from datetime import datetime
from peft import PeftModel

In [3]:
#!pip install -q --upgrade bitsandbytes peft trl accelerate

In [3]:
#!pip install vllm

ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=3.8; 0.1.0 Requires-Python >=3.8; 0.1.1 Requires-Python >=3.8; 0.1.2 Requires-Python >=3.8; 0.1.3 Requires-Python >=3.8; 0.1.4 Requires-Python >=3.8; 0.1.5 Requires-Python >=3.8; 0.1.6 Requires-Python >=3.8; 0.1.7 Requires-Python >=3.8; 0.2.0 Requires-Python >=3.8; 0.2.1 Requires-Python >=3.8; 0.2.1.post1 Requires-Python >=3.8; 0.2.2 Requires-Python >=3.8; 0.2.3 Requires-Python >=3.8; 0.2.4 Requires-Python >=3.8; 0.2.5 Requires-Python >=3.8; 0.2.6 Requires-Python >=3.8; 0.2.7 Requires-Python >=3.8; 0.3.0 Requires-Python >=3.8; 0.3.1 Requires-Python >=3.8; 0.3.3 Requires-Python >=3.8; 0.4.1 Requires-Python >=3.8; 0.4.2 Requires-Python >=3.8; 0.4.3 Requires-Python >=3.8; 0.5.0 Requires-Python >=3.8; 0.5.0.post1 Requires-Python >=3.8
ERROR: Could not find a version that satisfies the requirement vllm (from versions: none)
ERROR: No matching distribution found for vllm


In [ ]:
os.getcwd()

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch

In [ ]:
classified_krishna_df = pd.read_csv('/home/necuser/sdp/Classification/LLM/Finetuining/data/train/processed_chunked_files_llm_finetuining_krishna.csv')
classified_sandhya_df = pd.read_csv('/home/necuser/sdp/Classification/LLM/Finetuining/data/train/processed_chunked_files_llm_finetuining_sandhya.csv')


In [ ]:
classified_krishna_df

In [ ]:
# Concatenating the DataFrames row-wise
classified_df = pd.concat([classified_sandhya_df, classified_krishna_df], axis=0, ignore_index=True)

In [ ]:
classified_df['notes_character_count'] = classified_df['notes'].str.len()

In [ ]:


base_model_id = "BioMistral/BioMistral-7B"
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_eos_token=True,
    add_bos_token=True,
)


In [ ]:
classified_df['token_count_notes'] = classified_df['notes'].apply(lambda x: len(tokenizer.encode(x)))
classified_df['token_count_classified'] = classified_df['classified'].apply(lambda x: len(tokenizer.encode(x)))

In [ ]:
classified_df['num_split_notes'] = np.ceil(classified_df['token_count_notes']/1024).astype(int)

In [ ]:
print(classified_df[classified_df["filename"]=="Notes_text_7202_chunk_1.txt"]["notes"])

In [ ]:
len(classified_df)

In [ ]:
# Plotting the distribution using seaborn
plt.figure(figsize=(10, 6))
sns.histplot(classified_df['token_count_notes'], bins=10, kde=True)
plt.title('Distribution of Token Count')
plt.xlabel('Token Count')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(classified_df)

In [ ]:
dataset

In [ ]:
dataset = dataset.train_test_split(0.1)

In [ ]:
dataset

In [ ]:
def tokenize(prompt):
    result = tokenizer(prompt)
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""Classify the notes into below categories:
                   'allergies',
                   'chief_complaints',
                   'diagnosis',
                   'family_history',
                   'history',
                   'instructions_advice',
                   'investigation_report',
                   'investigations',
                   'medicine_prescription',
                   'observations_examinations',
                   'patient_willingness_concent',
                   'personal_history',
                   'procedure_report',
                   'referral',
                   'social_history',
                   'tolerance',
                   'treatment_plan',
                   'unclassified',
                   'vitals'
                    Output the classified data into json format


### notes:
{data_point["notes"]}

### Classified:
{data_point["classified"]}
"""
    return tokenize(full_prompt)

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = test_dataset.map(generate_and_tokenize_prompt)

In [ ]:
untokenized_text = tokenizer.decode(tokenized_train_dataset[1]['input_ids'])
print(untokenized_text)

In [ ]:
import matplotlib.pyplot as plt

def plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    lengths += [len(x['input_ids']) for x in tokenized_val_dataset]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()

plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

In [ ]:
max_length = 2048 #1024 #20000 # This was an appropriate max length for my dataset

# redefine the tokenize function and tokenizer

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token


def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = test_dataset.map(generate_and_tokenize_prompt)

In [ ]:
untokenized_text = tokenizer.decode(tokenized_train_dataset[1]['input_ids'])
print(untokenized_text)

In [ ]:
plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

In [ ]:
print("Notes: " + test_dataset[2]['notes'] + "\n")
print("Classified: " + test_dataset[2]['classified'] + "\n")

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

In [ ]:
# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

In [ ]:
# Re-init the tokenizer so it doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print(model)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

In [ ]:
print(model)

In [ ]:
import wandb, os
wandb.login()

In [ ]:
wandb_project = "biomistral-finetune-identifier-chunked"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

In [ ]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
import transformers
from datetime import datetime

project = "snp-finetune-classifier-2024_130624_v2"
base_model_name = "biomistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        max_steps=1000,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=50,
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        gradient_checkpointing_kwargs={'use_reentrant': True},
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "BioMistral/BioMistral-7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "biomistral-snp-finetune-classifier-2024/checkpoint-250")

In [ ]:
dataset

In [ ]:
eval_prompt = f"""Classify the notes into below categories:
                   'allergies',
                   'chief_complaints',
                   'diagnosis',
                   'family_history',
                   'history',
                   'instructions_advice',
                   'investigation_report',
                   'investigations',
                   'medicine_prescription',
                   'observations_examinations',
                   'patient_willingness_concent',
                   'personal_history',
                   'procedure_report',
                   'referral',
                   'social_history',
                   'tolerance',
                   'treatment_plan',
                   'unclassified',
                   'vitals'
                    Output the classified data into json format


### notes:
{dataset["test"]["notes"][1]}

### Classified:
"""
print(eval_prompt)

In [ ]:
dataset["test"]["classified"][1]

In [ ]:
%%time
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=2048)[0], skip_special_tokens=True))

In [ ]:
%%time
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=1024)[0], skip_special_tokens=True))

In [ ]:
%%time
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=512)[0], skip_special_tokens=True))

In [ ]:
import os
os.environ["WANDB_PROJECT"]="openhathi_instruct_finetuning"

from enum import Enum
from functools import partial
import pandas as pd
import torch

from transformers import AutoModelForCausalLM, LlamaTokenizer, TrainingArguments, set_seed
from datasets import load_dataset
from trl import SFTTrainer
from peft import get_peft_model, LoraConfig, TaskType

seed = 42
set_seed(seed)

In [ ]:
dataset_name = "smangrul/hinglish_self_instruct_v0"
dataset = load_dataset(dataset_name)
dataset

In [ ]:
dataset["train"][0]

In [ ]:
model_name = "sarvamai/OpenHathi-7B-Hi-v0.1-Base"
dataset_name = "smangrul/hinglish_self_instruct_v0"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
template = """{% for message in messages %}\n{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% if loop.last and add_generation_prompt %}{{'<|im_start|>assistant\n' }}{% endif %}{% endfor %}"""
tokenizer.chat_template = template

def preprocess(samples):
    batch = []
    for conversation in samples["messages"]:
        batch.append(tokenizer.apply_chat_template(conversation, tokenize=False))
    return {"content": batch}

dataset = load_dataset(dataset_name)
dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)
dataset = dataset["train"].train_test_split(0.1)
print(dataset)
print(dataset["train"][0])